# Module 5: Regime Prediction with Machine Learning - Part 1

In this part, we are going to explain the problem and get familiar with the dataset we are going to use.

## Table of Contents:
&nbsp;&nbsp;1. [Problem Description and Related Work](#1)


&nbsp;&nbsp;2. [Understand Data](#2)   

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2.0 [Set Up Environment](#2.0)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2.1 [Read Data and Description of the Variables](#2.1)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2.2 [Explanatory Data Anlaysis](#2.2)


&nbsp;&nbsp;3. [References](#3) 

## 1. Problem Description and Related Work <a id="1"></a>

Business cycle describes the rise and fall in the growth of the economy that occurs over time. Each business cyle has two turning points trough (or bottom) and peak. Expansion is measured from the trough of the previous business cycle to the peak of the current cycle, while recession is measured from the peak to the trough. A representation of business cycles is shown in __[figure](https://courses.lumenlearning.com/baycollege-introbusiness/chapter/reading-the-business-cycle-definition-and-phases/)__ below. In the United States (US), the Business Cycle Dating Committee of the National Bureau of Economic Research (NBER) determines the dates for business cycles. The turning points are determined by considering monthly growth indicators of the economy such as industrial production, employment, real income. The main focus of business cycle analysis is to analyze why economy goes through contraction and expansion periods. It is a well-studied topic in the literature and still an active reserach area. In our work, we are going to predict recessions in US economy with leading macroeconomic indicators using machine learning algorithms.

<center><img src="image2.png" align="center"  ></center>

Predicting business cycle turning points, especially economic recessions, is of great importance to investors, households, businesses in the economy.
Starting from &nbsp;[Mitchell and Burns (1938)](#a) pioneering work,  analyzing indicators of business cycles have become a core research area in business cycle analysis and there has been many work done in that field since then. Much of the work in the literature has indicated a wide range of economic and financial variables contain predictive information about future recessions in the economy.&nbsp;[Stock and Watson (1989)](#b) established coincedent and leading economic indicators for recession forecasting. &nbsp;[Estrella and Mishkin (1998)](#c) have documented predictive power of the slope of the term structure of Treasury yields and stock market for US recessions. 
&nbsp;[Liu and Moench (2016)](#f) have also showed the Treasury term spread has the highest predictive power with lagged abservations as well. Also, they indicated that balances in broker-dealer margin accounts have significant effect on recession predictions, especially in longer horizons.
&nbsp;[Chionis et al. (2009)](#d) worked on forecasting recessions in Europe with European Union (EU) data. They found out that the yield curve augmented with the composite stock index has significant predictive
power in terms of the EU real output.
&nbsp;[Ng (2014)](#e) worked on the problem through exploring effectiveness of boosting and found out interest rate measures and certain employment variables have predictive power for the recessions.
Recently, &nbsp;[Huang et al. (2018)](#g) have worked on predictive power of news sentiment analysis on recession forecasting in US economy. 



## 2. Understand Data <a id="2"></a>

### 2.0 Set Up Environment <a id="2.0"></a>

In [1]:
# load libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

### 2.1 Read Data and Description of the Variables <a id="2.1"></a>

For our analysis we will use a large macroeconomic database from FRED St. Louis designed by [McCracken and Ng (2015)](#i). It involves 129 macroeconomic monthly time series over the period 1959-2018. The data is organized into 8 categories (1)output and income, (2)labor market, (3)housing, (4)consumption, orders and inventories, (5)money and credit, (6)interest and exchange rates, (7)prices and (8)stock market. Detail description of the variables under each category can be found in __[appendix]( https://s3.amazonaws.com/files.fred.stlouisfed.org/fred-md/Appendix_Tables_Update.pdf)__.

In [8]:
import numpy as np #for numerical array data
import pandas as pd #for tabular data
import matplotlib.pyplot as plt #for plotting purposes

In [9]:
#bigmacro=pd.read_csv("Macroeconomic_VariablesL.csv")
bigmacro=pd.read_csv("current.csv")
bigmacro=bigmacro.rename(columns={'sasdate':'Date'})
bigmacro.tail()
bigmacro.head()

,Date,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
0,Transform:,5.000,5.0,5.000,5.0000,5.00000,5.0000,5.0000,5.0000,5.0000,...,6.000,6.000,6.00,6.00,6.00,2.0,6.0,6.0,6.0000,1.0
1,1/1/1959,2583.560,2426.0,15.188,276676.8154,18235.77392,21.9616,23.3868,22.2620,31.6664,...,18.294,10.152,2.13,2.45,2.04,NaN,6476.0,12298.0,84.2043,NaN
2,2/1/1959,2593.596,2434.8,15.346,278713.9773,18369.56308,22.3917,23.7024,22.4549,31.8987,...,18.302,10.167,2.14,2.46,2.05,NaN,6476.0,12298.0,83.5280,NaN
3,3/1/1959,2610.396,2452.7,15.491,277775.2539,18523.05762,22.7142,23.8459,22.5651,31.8987,...,18.289,10.185,2.15,2.45,2.07,NaN,6508.0,12349.0,81.6405,NaN
4,4/1/1959,2627.446,2470.0,15.435,283362.7075,18534.46600,23.1981,24.1903,22.8957,32.4019,...,18.300,10.221,2.16,2.47,2.08,NaN,6620.0,12484.0,81.8099,NaN


Consistent with the previous works in the literature, we use __[business cycle dating chronology provided by NBER](http://www.nber.org/cycles.html)__  which involves dates when recession began and ended in US economy. According to NBER's statistics we have 8 recession periods in our dataset where duration is changing from 6 to 18 months. We represent regimes as "Normal" and "Recession" in our dataset. 

In [10]:
Recession_periods=pd.read_csv('Recession_PeriodsL2024.csv')
bigmacro.insert(loc=1,column="Regime", value=Recession_periods['Regime'].values)
bigmacro.head()


,Date,Regime,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
0,Transform:,Normal,5.000,5.0,5.000,5.0000,5.00000,5.0000,5.0000,5.0000,...,6.000,6.000,6.00,6.00,6.00,2.0,6.0,6.0,6.0000,1.0
1,1/1/1959,Normal,2583.560,2426.0,15.188,276676.8154,18235.77392,21.9616,23.3868,22.2620,...,18.294,10.152,2.13,2.45,2.04,NaN,6476.0,12298.0,84.2043,NaN
2,2/1/1959,Normal,2593.596,2434.8,15.346,278713.9773,18369.56308,22.3917,23.7024,22.4549,...,18.302,10.167,2.14,2.46,2.05,NaN,6476.0,12298.0,83.5280,NaN
3,3/1/1959,Normal,2610.396,2452.7,15.491,277775.2539,18523.05762,22.7142,23.8459,22.5651,...,18.289,10.185,2.15,2.45,2.07,NaN,6508.0,12349.0,81.6405,NaN
4,4/1/1959,Normal,2627.446,2470.0,15.435,283362.7075,18534.46600,23.1981,24.1903,22.8957,...,18.300,10.221,2.16,2.47,2.08,NaN,6620.0,12484.0,81.8099,NaN


In [11]:
bigmacro[["Date","Regime"]].groupby("Regime").count()

,Date
Regime,
Normal,687
Recession,105


In [12]:
bigmacro.tail()

,Date,Regime,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
787,7/1/2024,Normal,19988.217,16306.3,120.966,1529744.0,710851.0,102.5340,100.5549,100.3012,...,119.776,127.972,31.20,35.70,27.96,66.4,548691.61,930374.96,5278.3525,14.4084
788,8/1/2024,Normal,20007.209,16322.1,121.052,1530317.0,710038.0,103.0831,101.0481,101.0128,...,119.653,128.291,31.26,35.81,27.97,67.9,551667.22,933066.90,5327.6461,19.6750
789,9/1/2024,Normal,20044.142,16333.7,121.690,1541305.0,716388.0,102.5283,100.1897,99.8299,...,119.220,128.682,31.44,36.00,28.11,70.1,553347.06,934283.59,5368.5924,17.6597
790,10/1/2024,Normal,20131.843,16394.9,121.837,1539243.0,719676.0,102.1123,99.5930,99.0546,...,119.073,129.148,31.57,36.22,28.18,70.5,554377.25,937299.96,5407.4120,19.9478
791,11/1/2024,Normal,20163.284,16433.7,122.173,NaN,724609.0,101.9621,99.5572,99.2239,...,119.124,129.363,31.63,36.22,28.30,71.8,NaN,NaN,5382.4182,15.9822


In [17]:
print('Number of recession periods in the dataset: ', Recession_periods[Recession_periods['Regime']=='Normal'].shape[0])
print('Number of expansion periods in the dataset: ', Recession_periods[Recession_periods['Regime']=='Recession'].shape[0])

Number of recession periods in the dataset:  687
Number of expansion periods in the dataset:  105


### 2.2 Explanatory Data Analysis <a id="2.2"></a>

Let's look at performance of stock market and economy over normal and recession periods. Below you can see the performance of SP500 and GDP over the period 1959-2018. Gray areas indicate recession periods in our dataset.

<img src="plots.png" />

## 3. References <a id="3"></a>

1. **W. Mitchell and A. Burns.** "Statistical Indicators of Cyclical Revivals", _National Bureau of Economic Research,_ 1938. <a class="anchor" id="a"></a>

2. **J. H. Stock and M. W. Watson.** "New Indexes of Coincedent and Leading Economic Indicators", _NBER Macroeconomics Annual,_ 1989. <a class="anchor" id="b"></a> 

3. **A. Estrella and F. S. Mishkin.** "Predicting U.S. Recessions: Financial Variables as Leading Indicators", _Review of Economics and Statistics,_ 1998. <a class="anchor" id="c"></a>

4. **D. Chionis, P. Gogas and I. Pragidis.** "Predicting Euroepan Union Recessions in the Euro Era: the Yield Curve as a Forecasting Tool of Economic Activity", _International Advances in Economic Research,_ 2009. <a class="anchor" id="d"></a>

5. **S. Ng.** "Viewpoint: Boosting Recessions", _Canadian Journal of Economics,_ 2014. <a class="anchor" id="e"></a>

6. **W. Liu and E. Moench** "What Predicts US Recessions?", _International Journal of Forecasting,_ 2016. <a class="anchor" id="f"></a>

7. **M. Y. Huang, R. R. Rojas and P. D. Convery.** " News Sentiment as Leading Indicators for Recessions", _arXiv,_ 2018. <a id="g"></a>

8. **M. McCracken and S. Ng** "__[FRED-MD: A Monthly Database for Macroeconomic Research](https://research.stlouisfed.org/econ/mccracken/fred-databases/)__", _Working Paper,_ 2015.  <a class="anchor" id="i"></a>


